# 🚆Train_station
지하철 정류장 데이터 전처리

### TODO
- [X] 데이터 불러오기, 필요없는 로우 및 컬럼 일차 삭제  
- [X] 결측값 확인
  - [X] 위경도 결측값 분석
  - [X] 결측값 채워진 데이터셋 구하기
  - [X] 노선명 통일
  - [X] 역사명 통일
- [X] 두 데이터셋 하나로 결합
  - [X] 역사명, 노선명 통일 
  - [X] 노선명을 지하철 호선명으로 통일, 중복역 확인
  - [X] 위경도 이상값 확인 
  - [X] 위경도 결측값 재확인
- [X] 지도에 시각화
  - [X] 이상값 확인
- [X] 역번호, 노선번호 맞추기
  - [X] 서울시 지하철 실시간 도착 정보에 쓰이는 역 종류 확인
  - [X] ~~전체 지하철 시간표 정보에 쓰이는 역 종류 확인~~
  - [X] 역사명 통일
  - [X] 노선명 통일
  - [X] 역번호 통일
  - [ ] ~~노선번호 통일~~
- [X] 노선도와 비교
- [ ] ~~서울 인접 역만 남기기~~
  - [X] 주소 데이터 결측값 확인
  - [X] 결측값 수동으로 채우기
  - [ ] ~~서울 인접 역만 남기고 자르기~~
  - [ ] ~~이상값 확인~~
- [X] 필요 없는 컬럼 제거
- [ ] 환승역, 종점, 분기역 정보 가공
- [X] 파일 생성
- [X] 역번호 맞추기
  - [X] 역번호 API로부터 가져오기
  - [X] station 파일과 맞추기
- [X] 역이름 맞추기
  - [X] station 파일과 맞추기
  - [X] API test
- [X] 세부 사항 변경
  - [X] 필요 없는 컬럼 제거
  - [X] 인코딩 변경

### 0. import

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from IPython.display import set_matplotlib_formats
import koreanize_matplotlib
import folium
import urllib.request
import json
import time

### 1. 데이터 불러오기, 필요없는 로우 및 컬럼 일차 삭제  

In [ ]:
# 데이터 읽어오기
station = pd.read_csv('train_station.csv')

# 확실히 필요없는 컬럼 제거
station = station.drop(columns=['영문역사명', '한자역사명', '운영기관명', '역사전화번호','데이터기준일자'])

station.rename(columns={'역사도로명주소': '주소'}, inplace=True)

# 확실히 필요없는 행 제거
# # 인천 1~2호선
# station = station[(station['노선번호'].str.slice(stop=3) != 'S28')]
# 인천 자기부상철도 -> 휴업 상태
station = station[(station['노선번호'] != 'S28M1')]
# 대전
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '30')]
# 대구 1~3호선
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '27')]
# 부산 1~4호선, 김해선
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '26') & (station['노선번호'] != 'L48B1')]
# 광주
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '29')]

# # 나중을 위해 역번호 저장
# station_id = station['역번호']

station = station.reset_index(drop=True)
station

### 2. 결측값 확인

##### 2-a. 위경도 결측값 분석

In [ ]:
# 역이 노선별로 총 몇 개인지 확인
# 1호선이어도, 구간별로 노선 명칭 다름 (우리가 아는 1호선 = 경원선 + 1호선 + 경인선 + 경부선 + 장항선)
total_station_cnt_by_route = station["노선명"].value_counts().sort_index()
total_station_cnt_by_route

In [ ]:
# 결측값이 컬럼별로 몇 개인지 확인
station.isnull().sum()

In [ ]:
# 역위도, 역경도가 결측값인 역이 노선별로 몇 개인지 확인
null_station_cnt_by_route = station['노선명'][station['역위도'].isnull(
)].value_counts().sort_index()
null_station_cnt_by_route

##### 2-b. 결측값 채워진 데이터셋 구하기

In [ ]:
# 1. 노선별 전체 역 개수가 실제 역 개수와 다름
# 2. 위경도 결측 데이터가 존재하는 노선에 대해 알아본 결과, 서울시 역사마스터 정보와 제대로 합처지지 않은 것으로 보임
# -> 서울시 역사마스터 정보와의 결합 필요

# 서울시 역사마스터 데이터 읽어오기
station_seoul = pd.read_csv('train_station_seoul.csv')

# 기존 데이터셋과 컬럼 이름 맞추기
station_seoul.rename(columns={'역사_ID': '역번호', '호선': '노선명'}, inplace=True)
station.rename(columns={'역위도': '위도', '역경도': '경도'}, inplace=True)

# 서울시 역사마스터 데이터, 위도와 경도가 반대로 들어가 있는 문제 해결
station_seoul.rename(columns={'위도': '경도', '경도': '위도'}, inplace=True)
station_seoul = station_seoul[['역번호', '역사명', '노선명', '위도', '경도']]

station_seoul

In [ ]:
# 역이 노선별로 몇 개인지 확인
total_station_seoul_cnt_by_route = station_seoul["노선명"].value_counts().sort_index()
total_station_seoul_cnt_by_route

# 확인 결과, 앞서 전체 역정보 데이터셋에 빠진 일부 역이 이 데이터셋에는 포함되어 있음 (역 개수 더 많음)

In [ ]:
# 역위도, 역경도가 결측값인 역이 노선별로 몇 개인지 확인
station_seoul.isnull().sum()

# 확인 결과, 해당 데이터셋은 위경도 정보가 빠진 것 없이 잘 들어있음 

##### 2-c. 노선명 통일

In [ ]:
# 둘 중 한 데이터셋에만 있는 노선명 확인
route_name = set(total_station_cnt_by_route.index)
route_name_seoul = set(total_station_seoul_cnt_by_route.index)
sub_route_name = route_name - route_name_seoul
sub_route_name_seoul = route_name_seoul - route_name

print(sub_route_name)
print(sub_route_name_seoul)

In [ ]:
# 노선명 통일
station["노선명"] = station["노선명"].replace({
    "도시철도 7호선": "7호선",
    "수도권  도시철도 9호선": "9호선",
    "수도권 경량도시철도 신림선": "신림선",
    "인천지하철 1호선": "인천1호선",
    "인천지하철 2호선": "인천2호선",
    "인천국제공항선": "공항철도선",
    "에버라인": "에버라인선",
    "의정부": "의정부선"
})

station_seoul["노선명"] = station_seoul["노선명"].replace({
    "공항철도1호선": "공항철도선",
    "안산선": "안산과천선",
    "과천선": "안산과천선",
    "9호선(연장)": "9호선",
    "7호선(인천)": "7호선",
    "신분당선(연장)": "신분당선",
    "신분당선(연장2)": "신분당선"
})

# 둘 중 한 데이터셋에만 있는 노선명 재확인
route_name = set(station["노선명"])
route_name_seoul = set(station_seoul["노선명"])
sub_route_name = route_name - route_name_seoul
sub_route_name_seoul = route_name_seoul - route_name

print(sub_route_name)
print(sub_route_name_seoul)

# 장항선, 중앙선은 이후 역 종류 보고 어디에 넣을지 결정하기 

In [ ]:
# 각 노선별 역 개수 확인해 보기
total_station_cnt_by_route = station["노선명"].value_counts().sort_index()
total_station_cnt_by_route

In [ ]:
total_station_seoul_cnt_by_route = station_seoul["노선명"].value_counts().sort_index()
total_station_seoul_cnt_by_route

##### 2-d. 역사명 통일

In [ ]:
# 둘 중 한 데이터셋에만 있는 역사명 확인
station_name = set(station["역사명"])
station_name_seoul = set(station_seoul["역사명"])
sub_station_name = sorted(station_name - station_name_seoul)
sub_station_name_seoul = sorted(station_name_seoul - station_name)

print(sub_station_name)
print(sub_station_name_seoul)

In [ ]:
# 역이름 통일
# 역이름 맨 끝에 등장하는'역'자 제거
station['역사명'] = station['역사명'].str.replace(pat=r'역$', repl=r'', regex=True)
# 서울역은 다시 '역' 추가 처리 
station['역사명'] = station['역사명'].str.replace(pat=r'^서울$', repl=r'서울역', regex=True)

# 결과 다시 확인
station_name = set(station["역사명"])
station_name_seoul = set(station_seoul["역사명"])
sub_station_name = sorted(station_name - station_name_seoul)
sub_station_name_seoul = sorted(station_name_seoul - station_name)

print(sub_station_name)
print(sub_station_name_seoul)

In [ ]:
# 특수문자 .으로 통일
station['역사명'] = station['역사명'].str.replace(pat=r'[·∙-]', repl=r'.', regex=True)
station_seoul['역사명'] = station_seoul['역사명'].str.replace(
    pat=r'[·∙-]', repl=r'.', regex=True)

# 수동 맞추기 -> 괄호 생기는 방향으로 (단, 지하/지상 정보, 노선명인 경우 제거)
station["역사명"] = station["역사명"].replace({
    "동작": "동작(현충원)",
    "검단오류": "검단오류(검단산업단지)",
    "관악산": "관악산(서울대)",
    "서울역(경의선)": "서울역",
    "시민공원": "시민공원(문화창작지대)",
    "수원(분당)": "수원",
    "신촌(지하)": "신촌",
    "운연": "운연(서창)",
    "흑석": "흑석(중앙대입구)",
    "아시아드경기장": "아시아드경기장(공촌사거리)",
})

station_seoul["역사명"] = station_seoul["역사명"].replace({
    "낙성대": "낙성대(강감찬)",
    "동대문역사문화공원": "동대문역사문화공원(DDP)",
    "동백": "동백(용인세브란스)",
    "운동장.송담대": "운동장.송담대(중앙시장)",
    "용마산": "용마산(용마폭포공원)",
})

# 결과 다시 확인
station_name = set(station["역사명"])
station_name_seoul = set(station_seoul["역사명"])
sub_station_name = sorted(station_name - station_name_seoul)
sub_station_name_seoul = sorted(station_name_seoul - station_name)

print(sub_station_name)
print(sub_station_name_seoul)

### 3. 두 데이터셋 하나로 결합

##### 3-a. 역사명, 노선명 하나로 합치기

In [ ]:
# 일단 하나로 합쳐보기
station_result = pd.merge(station, station_seoul, on=['역사명', '노선명'], how='outer')
station_result

In [ ]:
# 역번호_x가 null인 값이 노선별로 몇 개 있는지 각각 확인해보기
null_x_station_cnt_by_route = station_result['노선명'][station_result['역번호_x'].isnull(
)].value_counts().sort_index()
null_x_station_cnt_by_route

In [ ]:
# 역번호_y가 null인 값이 노선별로 몇 개 있는지 각각 확인해보기
null_y_station_cnt_by_route = station_result['노선명'][station_result['역번호_y'].isnull(
)].value_counts().sort_index()
null_y_station_cnt_by_route

In [ ]:
# 5호선
station_result[(station_result['노선명'] == '5호선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

# 역사명 합치기 
station_result.drop(785, inplace=True)
station_result.iloc[303, [10, 11, 12]] = (
    2535.0, 37.57254, 126.99030)

# 결과 확인
# 모든 역사명을 종로3가(탑골공원)로 맞추기
station_result["역사명"] = station_result["역사명"].replace({
    "종로3가": "종로3가(탑골공원)"})
station_result[station_result['역사명'] == '종로3가(탑골공원)']

In [ ]:
# 7호선
station_result[(station_result['노선명'] == '7호선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

# 역사명 합치기
station_result.drop(784, inplace=True)
station_result.iloc[403, [10, 11, 12]] = (
    2738.0, 37.485196, 126.981605)

# 결과 확인
# 모든 역사명 맞추기
station_result["역사명"] = station_result["역사명"].replace({
    "이수": "총신대입구(이수)",
    "총신대입구": "총신대입구(이수)"})
station_result[station_result['역사명'] == '총신대입구(이수)']

In [ ]:
# 9호선
station_result[(station_result['노선명'] == '9호선') & (station_result['역번호_x'].isnull())]

# 맞출 데이터 없음

In [ ]:
# 경부선
print(len(station_result[(station_result['노선명'] == '장항선') & (
    station_result['역번호_x'].isnull())]))
station_result[(station_result['노선명'] == '경부선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

# 나머지는 장항선과 합치기

In [ ]:
# 서울역 데이터 수정 필요
station_result.drop([558, 823], inplace=True)
station_result[station_result['역사명'] == '서울역']

In [ ]:
# 장항선
print(len(station_result[(station_result['노선명'] == '장항선') & (
    station_result['역번호_x'].isnull())]))
station_result[(station_result['노선명'] == '장항선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

# 역사명 합치기
station_result.drop([788, 789, 790, 791, 792, 793, 794] , inplace=True)
station_result.iloc[504, [10, 11, 12]] = (
    1401.0, 36.801215, 127.135763)
station_result.iloc[505, [10, 11, 12]] = (
    1402.0, 36.793759, 127.121400)
station_result.iloc[506, [10, 11, 12]] = (
    1403.0, 36.792053, 127.104361)
station_result.iloc[507, [10, 11, 12]] = (
    1404.0, 36.788660, 127.084850)
station_result.iloc[508, [10, 11, 12]] = (
    1405.0, 36.777629, 127.052991)
station_result.iloc[509, [10, 11, 12]] = (
    1407.0, 36.780483, 127.003249)
station_result.iloc[510, [10, 11, 12]] = (
    1408.0, 36.769502, 126.951108)

station_result[(station_result['노선명'] == '장항선')]

In [ ]:
# 바뀐 값 확인
station_result.iloc[504:511]

In [ ]:
# 경인선
station_result[(station_result['노선명'] == '경인선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

# 역사명 합치기
station_result.drop(786, inplace=True)
station_result.iloc[620, [10, 11, 12]] = (
    1821.0, 37.492433, 126.824086)

# 결과 확인
# 모든 역사명 맞추기
station_result["역사명"] = station_result["역사명"].replace({
    "온수": "온수(성공회대입구)"})
station_result[station_result['역사명'] == "온수(성공회대입구)"]

In [ ]:
# 경춘선
station_result[(station_result['노선명'] == '경춘선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

In [ ]:
# 분당선
station_result[(station_result['노선명'] == '분당선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

In [ ]:
# 서해선
station_result[(station_result['노선명'] == '서해선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

# 원곡역 -> 시우역으로 이름 변화함
station_result.drop(768, inplace=True)
station_result.iloc[766, [10, 11, 12]] = (
    4814.0, 37.31321, 126.796261)

station_result[(station_result['노선명'] == '서해선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

In [ ]:
# 안산과천선
station_result[(station_result['노선명'] == '안산과천선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

In [ ]:
# 수인선
station_result[(station_result['노선명'] == '수인선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

In [ ]:
# 에버라인선
station_result[(station_result['노선명'] == '에버라인선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

# 역사명 합치기
station_result.drop(770, inplace=True)
station_result.iloc[77, [10, 11, 12]] = (
    4501.0, 37.275449, 127.116665)

# 결과 확인
# 모든 역사명 맞추기
station_result["역사명"] = station_result["역사명"].replace({
    "기흥": "기흥(백남준아트센터)"})
station_result[station_result['역사명'] == "기흥(백남준아트센터)"]


In [ ]:
# 우이신설선
station_result[(station_result['노선명'] == '우이신설선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

# 역사명 합치기
station_result.drop(769, inplace=True)
station_result.iloc[116, [10, 11, 12]] = (
    4711.0, 37.592467, 127.016516)

# 결과 확인
# 모든 역사명 맞추기
station_result["역사명"] = station_result["역사명"].replace({
    "성신여대입구": "성신여대입구(돈암)",
    "돈암": "성신여대입구(돈암)"})
station_result[station_result['역사명'] == "성신여대입구(돈암)"]

In [ ]:
# 경원선
print(len(station_result[(station_result['노선명'] == '경원선') & (
    station_result['역번호_x'].isnull())]))
station_one = station_result[(station_result['노선명'] == '경원선') & (
    station_result['역번호_x'].isnull())]

station_one = station_one.replace({'경원선': '경의중앙선'})

station_one

In [ ]:
# 중앙선
print(len(station_result[(station_result['노선명'] == '중앙선') & (
    station_result['역번호_x'].isnull())]))
station_cau = station_result[(station_result['노선명'] == '중앙선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

station_cau = station_cau.replace({'중앙선': '경의중앙선'})

station_cau

In [ ]:
# 경의중앙선
print(len(station_result[(station_result['노선명'] == '경의중앙선') & (
    station_result['역번호_y'].isnull())]))
station_gyeong = station_result[(station_result['노선명'] == '경의중앙선') & (
    station_result['역번호_x'].isnull() | station_result['역번호_y'].isnull())]

# y 컬럼 제거
station_gyeong = station_gyeong.drop(columns=['역번호_y', '위도_y', '경도_y'])

# 역이름 변경
station_gyeong = station_gyeong.replace({
    '청량리': '청량리(서울시립대입구)',
    '왕십리': '왕십리(성동구청)',
    '이촌': '이촌(국립중앙박물관)',
    '상봉': '상봉(시외버스터미널)'
})

station_result = station_result.replace({
    '청량리': '청량리(서울시립대입구)',
    '왕십리': '왕십리(성동구청)',
    '이촌': '이촌(국립중앙박물관)',
    '상봉': '상봉(시외버스터미널)'
})

station_gyeong

In [ ]:
# 경원선 + 중앙선 = 경의중앙선 으로 합치기
station_temp = pd.concat([station_one, station_cau], axis = 0)

# x 컬럼 제거
station_temp = station_temp.drop(columns=['역번호_x', '노선번호', '환승역구분', '환승노선번호', '환승노선명', '주소', '위도_x', '경도_x'])

# merge
station_temp = pd.merge(station_gyeong, station_temp, on=['역사명', '노선명'], how='outer')

station_temp

In [ ]:
# 기존 데이터에 추가, 기존에 있던 애들은 삭제
station_result.drop(index=station_one.index, inplace=True)
station_result.drop(index=station_cau.index, inplace=True)
station_result.drop(index=station_gyeong.index, inplace=True)
station_result = pd.concat([station_result, station_temp])

station_result = station_result.reset_index(drop=True)
station_result

##### 3-b. 노선명을 지하철 호선명으로 통일, 중복값 확인

In [ ]:
# 노선명 종류 확인
station_cnt_by_route = station_result['노선명'].value_counts().sort_index()
station_cnt_by_route

In [ ]:
# 경부선, 경원선, 경인선 = 1호선
# 분당선 + 수인선 = 수인분당선
# 일산선 = 3호선
# 진접선 = 4호선

station_result["노선명"] = station_result["노선명"].replace({
    "경부선": "1호선",
    "경원선": "1호선",
    "경인선": "1호선",
    "분당선": "수인분당선",
    "수인선": "수인분당선",
    "일산선": "3호선",
    "안산과천선": "4호선", 
    "진접선": "4호선"
})

station_result["노선명"] = station_result["노선명"].replace({
    "수인수인분당선": "수인분당선"
})

# 노선명 종류 재확인
station_cnt_by_route = station_result['노선명'].value_counts().sort_index()
station_cnt_by_route

In [ ]:
# 노선별 중복 확인
station_result[station_result.duplicated(['노선명', '역사명'])]

In [ ]:
# 7호선 데이터 확인
station_result.iloc[57:75]

# 역번호_y, 위도, 경도 정보가 서로 달라 잘못 저장됨
# 확인 결과, 2로 시작하는 쪽의 역번호가 틀림 (짝수 index 행들)

In [ ]:
# 5호선 데이터 확인
station_result.iloc[329:333]

# 역번호, 위도, 경도 정보가 달라 잘못 저장됨
# 확인 결과, 9로 시작하는 쪽의 역번호가 틀림 (홀수 index 행들)

In [ ]:
# 경의중앙선 데이터 확인
station_result[(station_result["역사명"] == "공덕") |
               (station_result["역사명"] == "홍대입구") | (station_result["역사명"] == "디지털미디어시티")]

# 확인 결과, 어차피 경중선 역코드는 필요 없음... 1-8호선이 아니므로 
# 그냥 같은 쪽으로 일치시키는 방향으로 변화시키기 

In [ ]:
# 3호선 데이터 확인
station_result[(station_result["역사명"] == "지축")]

# 확인 결과, 위쪽이 맞음 

In [ ]:
station_result.drop([58, 60, 62, 64, 66, 68, 70, 72, 74, 329, 331, 562, 565, 567, 714], inplace=True)

# 재확인
station_result[station_result.duplicated(['노선명', '역사명'])]

##### 3-c. 위경도값 하나로 합치기

In [ ]:
# x, y에 대해, 둘 중 한 곳에 없는 위경도값이 각각 몇 개인지 count
lat_in_only_x = station_result[station_result["위도_y"].isnull(
) & station_result["위도_x"].notnull()]
lng_in_only_x = station_result[station_result["경도_y"].isnull(
) & station_result["경도_x"].notnull()]

lat_in_only_x_cnt = len(lat_in_only_x)
lng_in_only_x_cnt = len(lng_in_only_x)

print("x에만 있는 위경도값 개수 :", lat_in_only_x_cnt, lng_in_only_x_cnt)

lat_in_only_y = station_result[station_result["위도_x"].isnull(
) & station_result["위도_y"].notnull()]
lng_in_only_y = station_result[station_result["경도_x"].isnull(
) & station_result["경도_y"].notnull()]

lat_in_only_y_cnt = len(lat_in_only_y)
lng_in_only_y_cnt = len(lng_in_only_y)

print("y에만 있는 위경도값 개수 :", lat_in_only_y_cnt, lng_in_only_y_cnt)

# 한쪽 위경도가 NaN이면, 반대쪽 위경도 넣어주기
station_result["위도_x"] = station_result["위도_x"].fillna(station_result["위도_y"])
station_result["위도_y"] = station_result["위도_y"].fillna(station_result["위도_x"])
station_result["경도_x"] = station_result["경도_x"].fillna(station_result["경도_y"])
station_result["경도_y"] = station_result["경도_y"].fillna(station_result["경도_x"])

# 위경도 데이터의 오차 계산
station_result["위도차"] = (station_result["위도_x"] - station_result["위도_y"]).abs()
station_result["경도차"] = (station_result["경도_x"] - station_result["경도_y"]).abs()

In [ ]:
# 위도 오차값 확인
lat_diff_mean = station_result["위도차"].mean()
print(lat_diff_mean)

# 평균보다 오차 큰 행 확인
bigger_lat_diff = station_result[station_result["위도차"] >= lat_diff_mean]
bigger_lat_diff

In [ ]:
# 경도 오차값 확인
lng_diff_mean = station_result["경도차"].mean()
print(lng_diff_mean)

# 평균보다 오차 큰 행 확인
bigger_lng_diff = station_result[station_result["경도차"] >= lng_diff_mean]
bigger_lng_diff

In [ ]:
# 119 ~ 134 위도_x와 경도_x가 뒤바뀌어 들어감
# 송도, (위도_x, 경도_x) 전남 해남에 위치
# 남동구청, (위도_x, 경도_x) 황해에 위치
# 양원, (위도_x, 경도_x) 양원역 위치가 경북 봉화군에 위치한 양원역 위치로 잘못 들어가져 있음
# -> 위경도 데이터의 경우, x를 y로 덮어쓰면 될 것 같음
# 이를 증명하기 위해, 위의 데이터들의 위도차/경도차가 해결된 뒤,
# 위경도차가 높은 행들을 각각 확인하여, 거리상의 오차가 어느 정도일지 확인해볼 것

station_result.loc[[119, 120, 121, 122, 123, 124, 125, 126,
                    127, 128, 129, 130, 131, 132, 133, 134, 688, 54, 758], ['위도차', '경도차']] = 0.0

In [ ]:
# 위도 오차값 평균 재확인
lat_diff_mean = station_result["위도차"].mean()
print(lat_diff_mean)

# 평균보다 오차 큰 행 확인
bigger_lat_diff = station_result[station_result["위도차"] >= lat_diff_mean]
bigger_lat_diff.sort_values('위도차').tail()

In [ ]:
# 경도 오차값 평균 재확인
lng_diff_mean = station_result["경도차"].mean()
print(lng_diff_mean)

# 평균보다 오차 큰 행 확인
bigger_lng_diff = station_result[station_result["경도차"] >= lng_diff_mean]
bigger_lng_diff.sort_values('경도차').tail()

##### 3-b. 위경도 결측값 재확인

In [ ]:
# 구리, (위도_x, 경도_x) 구로 들어가 있음
# 화전, (위도_x, 경도_x) 덕양구청에 위치, 도보 1시간 30분의 오차
# 그 다음으로 오차가 큰 인천공항2터미널과 서울역의 경우 역이 워낙 커서 생기는 일, 문제 없음
# 즉, x와 y 두 곳에 있는 값들에 대해서는 이상값 검토가 끝남 
# 둘 다에 있는 값의 경우 그냥 y쪽 값으로 덮어쓰기
# 둘 중 한 곳에만 있는 위경도값들은 위에서 따로 저장해 둠, 
# 이후 지도를 보며 재검토할 예정  

# y에 없는 값들 x에서 가져와 채우기
station_result['위도_y'] = station_result['위도_y'].fillna(station_result['위도_x'])
station_result['경도_y'] = station_result['경도_y'].fillna(station_result['경도_x'])

# 개수 재확인
null_lat_in_y = station_result[station_result["위도_y"].isnull()]
null_lng_in_y = station_result[station_result["경도_y"].isnull()]
null_lat_in_y_cnt = len(null_lat_in_y)
null_lng_in_y_cnt = len(null_lng_in_y)

print("y에 있는 위경도 결측값 개수 :", null_lat_in_y_cnt, null_lng_in_y_cnt)

null_lat_in_y 

In [ ]:
# 위도_x, 경도_x 삭제 및 위도_y와 경도_y의 이름 변경
station_result.drop(columns=["위도_x", "경도_x", "위도차", "경도차"], inplace=True)
station_result.rename(columns={'위도_y': '위도', '경도_y': '경도'}, inplace=True)

station_result

### 4. 지도에 시각화

##### 4-a. 이상값 확인

In [ ]:
# 지도 시각화 
geo_station = station_result.copy()

# 중심 설정
fmap = folium.Map(location=[geo_station['위도'].mean(
), geo_station['경도'].mean()], zoom_start=0, width=750, height=500)

for n in geo_station.index:
    # 팝업에 들어갈 텍스트를 지정
    popup_name = str(geo_station.loc[n, '위도']) + ", " + str(geo_station.loc[n, '경도']) + \
        " \n" + geo_station.loc[n, '역사명'] + ' - ' + geo_station.loc[n, '노선명']
    
    folium.Marker(
        location=[geo_station.loc[n, '위도'], geo_station.loc[n, '경도']],
        popup=popup_name, 
        tooltip=popup_name
    ).add_to(fmap)

fmap

In [ ]:
# 위경도가 한국을 벗어난 역들 찾아보기
station_outof_korea = station_result[(station_result['위도'] < 33.10000000) |
                                     (station_result['위도'] > 38.45000000) | (station_result['경도'] < 124.19583333) |
                                     (station_result['경도'] > 131.87222222)]

station_outof_korea

# 확인 결과, 모든 역이 제대로 한국 내에 존재함

In [ ]:
# 더 크게 지도 시각화 
geo_station = station_result.copy()

# 중심 설정
fmap = folium.Map(location=[geo_station['위도'].mean(
), geo_station['경도'].mean()], zoom_start=8, width=750, height=500)

for n in geo_station.index:
    # 팝업에 들어갈 텍스트를 지정
    popup_name = str(geo_station.loc[n, '위도']) + ", " + str(geo_station.loc[n, '경도']
                                                            ) + " \n" + geo_station.loc[n, '역사명'] + ' - ' + geo_station.loc[n, '노선명']

    folium.Marker(
        location=[geo_station.loc[n, '위도'], geo_station.loc[n, '경도']],
        popup=popup_name,
        tooltip=popup_name
    ).add_to(fmap)

fmap

### 5. 역번호, 노선번호, 역이름, 노선명 맞추기

##### 5-a. 서울시 지하철 실시간 도착정보에 쓰이는 역 및 노선 종류 확인

In [ ]:
# 데이터 읽어오기
station_num_name_seoul = pd.read_csv('train_station_num_name_seoul.csv')

# 컬럼명 변경
station_num_name_seoul.columns=["역번호", "역사명", "노선명", "외부코드"]

station_num_name_seoul

In [ ]:
# 결측값 존재 확인
station_num_name_seoul.info()

# 확인 결과, 외부코드를 제외한 모든 값이 잘 들어있음 

In [ ]:
# 해당 데이터에 포함된 노선명 확인 및 둘 중 한 데이터셋에만 있는 노선명 확인
route_name_result = set(station_result['노선명'])
route_name_seoul = set(station_num_name_seoul["노선명"])
sub_route_name_result = route_name_result - route_name_seoul
sub_route_name_seoul = route_name_seoul - route_name_result

print(route_name_result)
print(route_name_seoul)

print(sub_route_name_result)
print(sub_route_name_seoul)

# 확인 결과, 공항철도, 경춘선 이름을 맞출 필요 있음
# 수인분당선, 경의선의 경우 다른, 다른 모든 API 확인한 뒤 통합할지 쪼갤지 고민 필요

In [ ]:
# 일단 맞출 수 있는 부분은 통일
station_num_name_seoul["노선명"] = station_num_name_seoul["노선명"].replace({
    "우이신설경전철": "우이신설선",
    "의정부경전철": "의정부선",
    "용인경전철": "에버라인선",
    "김포도시철도": "김포골드라인",
    "인천선": "인천1호선",
    "공항철도": "공항철도선"
})
station_num_name_seoul["노선명"] = station_num_name_seoul["노선명"].str.replace(
    pat=r'^0', repl=r'', regex=True)

# 둘 중 한 데이터셋에만 있는 노선명 재확인
route_name_result = set(station_result['노선명'])
route_name_seoul = set(station_num_name_seoul["노선명"])
sub_route_name_result = route_name_result - route_name_seoul
sub_route_name_seoul = route_name_seoul - route_name_result

print(route_name_result)
print(route_name_seoul)

print(sub_route_name_result)
print(sub_route_name_seoul)

In [ ]:
# 역사명 맞추기
station_name_result = set(station_result['역사명'])
station_name_seoul = set(station_num_name_seoul['역사명'])
sub_station_name_result = station_name_result - station_name_seoul
sub_station_name_seoul = station_name_seoul - station_name_result

print(station_name_result)
print(station_name_seoul)

print(sub_station_name_result)
print(sub_station_name_seoul)

In [ ]:
# 수동으로 이름 맞추기
station_num_name_seoul["역사명"] = station_num_name_seoul["역사명"].replace({
    "이촌": "이촌(국립중앙박물관)", 
    "4?19민주묘지": "4.19민주묘지",
    "청량리": "청량리(서울시립대입구)",
    "양재": "양재(서초구청)",
    "경복궁": "경복궁(정부서울청사)",
    "동대문역사문화공원": "동대문역사문화공원(DDP)",
    "구의": "구의(광진구청)",
    "굽은다리": "굽은다리(강동구민회관앞)",
    "월드컵경기장": "월드컵경기장(성산)",
    "화랑대": "화랑대(서울여대입구)",
    "군자": "군자(능동)",
    "총신대입구": "총신대입구(이수)",
    "이수": "총신대입구(이수)",
    "시민공원": "시민공원(문화창작지대)",
    "운연": "운연(서창)",
    "남부터미널": "남부터미널(예술의전당)",
    "광교": "광교(경기대)",
    "회현": "회현(남대문시장)",
    "하남시청": "하남시청(덕풍.신장)",
    "새절": "새절(신사)",
    "증산": "증산(명지대앞)",
    "상월곡": "상월곡(한국과학기술연구원)",
    "녹사평": "녹사평(용산구청)",
    "석남": "석남(거북시장)",
    "수유": "수유(강북구청)",
    "양재시민의숲": "양재시민의숲(매헌)",
    "교대": "교대(법원.검찰청)",
    "삼성": "삼성(무역센터)",
    "아시아드경기장": "아시아드경기장(공촌사거리)",
    "봉화산": "봉화산(서울의료원)",
    "성신여대입구": "성신여대입구(돈암)",
    "종로3가": "종로3가(탑골공원)",
    "동작": "동작(현충원)",
    "오목교": "오목교(목동운동장앞)",
    "기흥": "기흥(백남준아트센터)",
    "남한산성입구": "남한산성입구(성남법원.검찰청)",
    "용마산": "용마산(용마폭포공원)",
    "광나루": "광나루(장신대)",
    "낙성대": "낙성대(강감찬)",
    "숭실대입구": "숭실대입구(살피재)",
    "숙대입구": "숙대입구(갈월)",
    "충정로": "충정로(경기대입구)",
    "가정": "가정(루원시티)",
    "검단오류": "검단오류(검단산업단지)",
    "대흥": "대흥(서강대앞)",
    "온수": "온수(성공회대입구)",
    "왕십리": "왕십리(성동구청)",
    "천호": "천호(풍납토성)",
    "잠실": "잠실(송파구청)",
    "강변": "강변(동서울터미널)",
    "광흥창": "광흥창(서강)",
    "고려대": "고려대(종암)",
    "흑석": "흑석(중앙대입구)",
    "관악산": "관악산(서울대)",
    "미아": "미아(서울사이버대학)",
    "공릉": "공릉(서울과학기술대)",
    "사우": "사우(김포시청)",
    "상봉": "상봉(시외버스터미널)",
    "용두": "용두(동대문구청)",
    "운동장.송담대": "운동장.송담대(중앙시장)",
    "광교중앙": "광교중앙(아주대)",
    "동백": "동백(용인세브란스)",
    "아차산": "아차산(어린이대공원후문)",
    "신정": "신정(은행정)",
    "대림": "대림(구로구청)",
    "월곡": "월곡(동덕여대)",
    "한성대입구": "한성대입구(삼선교)",
    "올림픽공원": "올림픽공원(한국체대)",
    "신창": "신창(순천향대)",
    "쌍용": "쌍용(나사렛대)",
    "광화문": "광화문(세종문화회관)",
    "안암": "안암(고대병원앞)",
    "몽촌토성": "몽촌토성(평화의문)",
    "서울대입구": "서울대입구(관악구청)",
    "어린이대공원": "어린이대공원(세종대)"
})

# 역사명 재확인
# station_name_result = set(station_result['역사명'])
station_name_seoul = set(station_num_name_seoul['역사명'])
sub_station_name_result = station_name_result - station_name_seoul
sub_station_name_seoul = station_name_seoul - station_name_result

print(station_name_result)
print(station_name_seoul)

print(sub_station_name_result)
print(sub_station_name_seoul)

In [ ]:
station_result[(station_result["역사명"] == "운천") |
               (station_result["역사명"] == "지제") | (station_result["역사명"] == "평택지제")]

# 지제 -> 평택지제 이름 변경
station_result["역사명"] = station_result["역사명"].replace({
    "지제": "평택지제"
})

station_result[(station_result["역사명"] == "운천") |
               (station_result["역사명"] == "지제") | (station_result["역사명"] == "평택지제")]

In [ ]:
station_num_name_seoul[(station_num_name_seoul["역사명"] == "운천") |
                       (station_num_name_seoul["역사명"] == "지제") | (station_num_name_seoul["역사명"] == "평택지제")]

# 수동으로 운천 역번호 추가
station_num_name_seoul.loc[767] = ['1286', '운천', '경의선', np.NaN]

station_num_name_seoul[(station_num_name_seoul["역사명"] == "운천") |
                       (station_num_name_seoul["역사명"] == "지제") | (station_num_name_seoul["역사명"] == "평택지제")]

In [ ]:
# 두 값 합치기
station_temp = pd.merge(station_result, station_num_name_seoul, on=['역사명', '노선명'], how='left')
station_temp = station_temp.astype({'역번호_y': 'str'})

station_temp = station_temp.reset_index(drop=True)
station_temp

In [ ]:
# 역번호_y 데이터형식 맞추기
station_temp = station_temp.astype({'역번호_y': 'str'})
station_temp['역번호_y'] = station_temp['역번호_y'].str.replace(
    pat=r'.0$', repl=r'', regex=True)

station_temp 

In [ ]:
# 역번호_y != 역번호인 노선 몇 개 있는지 각각 확인해보기
null_station_cnt_by_route = station_temp['노선명'][station_temp['역번호'] != station_temp['역번호_y']].value_counts().sort_index()
null_station_cnt_by_route

In [ ]:
# 1호선
station_temp[(station_temp['노선명'] == '1호선') & (station_temp['역번호'] != station_temp['역번호_y'])]

# 확인 결과, 역번호가 맞음

In [ ]:
# 5호선
station_temp[(station_temp['노선명'] == '5호선') & (
    station_temp['역번호'] != station_temp['역번호_y'])]

# 확인 결과, 역번호가 맞음 

In [ ]:
# 경의중앙
station_temp[(station_temp['노선명'] == '경의중앙선') & (station_temp['역번호'] != station_temp['역번호_y'])]

# 경중선은 아차피 역번호로 시간표 찾기 제공하지 않음

In [ ]:
# 경춘선
station_temp[(station_temp['노선명'] == '경춘선') & (station_temp['역번호'] != station_temp['역번호_y'])]

# 경춘선은 아차피 역번호로 시간표 찾기 제공하지 않음

In [ ]:
# 서해선
station_temp[(station_temp['노선명'] == '서해선') & (station_temp['역번호'] != station_temp['역번호_y'])]

# 서해선은 어차피 역번호로 시간표 제공하지 않음

In [ ]:
# 수인분당선
station_temp[(station_temp['노선명'] == '수인분당선') & (
    station_temp['역번호'] != station_temp['역번호_y'])]

# 수인분당선도 어차피 제공하지 않음

In [ ]:
# 역번호_y == 역번호 == NaN인 역 확인 <- X
station_temp[(station_temp['역번호'].isnull()) & (station_temp['역번호_y'].isnull())]

# 역번호 == NaN인 역 확인 <- 경의중앙선 
station_temp[(station_temp['역번호'].isnull())]

# 즉, 그냥 역번호_y 대신 역번호 사용하면 됨 
station_result = station_temp
station_result.drop(columns=["역번호_x", "역번호_y"], inplace=True)

station_result

In [ ]:
# 역번호 수정
station_result["역번호"][station_result["역번호"].str.len() < 4] = "0" + station_result["역번호"][station_result["역번호"].str.len() < 4]

# 확인 결과, 제대로 해결
station_result[station_result["역번호"].str.len() < 4]

### 6. 실제 노선도와 비교

In [ ]:
# 역이 노선별로 몇 개인지 확인
total_station_cnt_by_route = station_result["노선명"].value_counts(
).sort_index()
total_station_cnt_by_route

In [ ]:
# 1호선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "1호선"]))

In [ ]:
# 2호선
print(list(station_result["역사명"][station_result["노선명"] == "2호선"]))

# 2호선 까치산 없음, 추가 필요
station_result[station_result["역사명"] == "까치산"]

In [ ]:
# 역번호 찾아오기 = 0200
station_num_name_seoul[station_num_name_seoul["역사명"] == "까치산"]

# 정보 수동 입력해서 추가해주기
station_result.loc[766] = ["까치산", np.NaN, "2호선", np.NaN,
                           np.NaN, np.NaN, "서울특별시 강서구 강서로 54", 37.531394, 126.846987, "0200", np.NaN]

# 재확인
station_result[station_result["역사명"] == "까치산"]

In [ ]:
# 3호선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "3호선"]))

In [ ]:
# 4호선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "4호선"]))

In [ ]:
# 5호선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "5호선"]))

In [ ]:
# 6호선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "6호선"]))

In [ ]:
# 7호선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "7호선"]))

In [ ]:
# 8호선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "8호선"]))

In [ ]:
# 9호선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "9호선"]))

In [ ]:
# 경강선 - 모두 존재 
print(list(station_result["역사명"][station_result["노선명"] == "경강선"]))

In [ ]:
# 경의중앙선 
print(list(station_result["역사명"][station_result["노선명"] == "경의중앙선"]))

# 도라산 빠짐, 추가 필요
station_result[station_result["역사명"] == "도라산"]

In [ ]:
# 역번호 없음 (어차피 필요 없음)
station_num_name_seoul[station_num_name_seoul["역사명"] == "도라산"]

# 정보 수동 입력해서 추가해주기
station_result.loc[767] = ["도라산", np.NaN, "경의중앙선", np.NaN,
                           np.NaN, np.NaN, "경기도 파주시 장단면 희망로 307", 37.898307, 126.709193, np.NaN, np.NaN]

# 재확인
station_result[station_result["역사명"] == "도라산"]

In [ ]:
# 경춘선 
print(list(station_result["역사명"][station_result["노선명"] == "경춘선"]))

# 광운대 빠짐, 추가 필요
station_result[station_result["역사명"] == "광운대"]

In [ ]:
# 역번호 1305
station_num_name_seoul[station_num_name_seoul["역사명"] == "광운대"]

# 정보 수동 입력해서 추가해주기
station_result.loc[768] = ["광운대", np.NaN, "경춘선", np.NaN,
                           np.NaN, np.NaN, "서울특별시 노원구 석계로 98-2", 37.623986, 127.061956, "1305", np.NaN]

# 재확인
station_result[station_result["역사명"] == "광운대"]

In [ ]:
# 공항철도선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "공항철도선"]))

In [ ]:
# 김포골드라인 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "김포골드라인"]))

In [ ]:
# 서해선 - 모두 존재 
print(list(station_result["역사명"][station_result["노선명"] == "서해선"]))

In [ ]:
# 수인분당선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "수인분당선"]))

In [ ]:
# 신림선 - 모두 존재 
print(list(station_result["역사명"][station_result["노선명"] == "신림선"]))

In [ ]:
# 신분당선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "신분당선"]))

In [ ]:
# 에버라인선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "에버라인선"]))

In [ ]:
# 우이신설선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "우이신설선"]))

In [ ]:
# 의정부선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "의정부선"]))

In [ ]:
# 인천1호선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "인천1호선"]))

In [ ]:
# 인천2호선 - 모두 존재
print(list(station_result["역사명"][station_result["노선명"] == "인천2호선"]))

### 7. 서울 근교역 추리기

##### 7-a. 주소 결측값 확인

In [ ]:
null_address_station_cnt_by_route = station_result['노선명'][station_result['주소'].isnull(
)].value_counts().sort_index()

null_address_station_cnt_by_route

In [ ]:
station_result[station_result['주소'].isnull()]

In [ ]:
# 수동으로 추가
station_result.iloc[722, 6] = '서울특별시 강동구 둔촌동 8-1'
station_result.iloc[723, 6] = '서울특별시 강동구 둔촌동 227-7'
station_result.iloc[724, 6] = '서울특별시 송파구 방이동 89-28'
station_result.iloc[725, 6] = '서울특별시 송파구 방이동 88-17'
station_result.iloc[726, 6] = '서울특별시 송파구 방이동 2'
station_result.iloc[727, 6] = '서울특별시 송파구 석촌동 209'
station_result.iloc[728, 6] = '서울특별시 송파구 삼전동 157-1'
station_result.iloc[729, 6] = '서울특별시 송파구 잠실동 347'
station_result.iloc[730, 6] = '서울특별시 송파구 잠실동 123'
station_result.iloc[731, 6] = '서울특별시 강남구 삼성동 172'
station_result.iloc[732, 6] = '서울특별시 강남구 삼성동 111-147'
station_result.iloc[733, 6] = '서울특별시 강남구 삼성동 111-114'
station_result.iloc[734, 6] = '서울특별시 강남구 논현동 279-165'
station_result.iloc[735, 6] = '경기도 군포시 군포로 750'

# 확인 결과, 주소 결측값 없음
station_result[station_result['주소'].isnull()]

### 8. 필요 없는 컬럼 제거 

In [ ]:
station_result.drop(columns=["노선번호", "환승역구분", "환승노선번호", "환승노선명"], inplace=True)
station_result

### 9. 파일 생성

In [ ]:
station_result.to_csv("result_train_station.csv")

### 10. 역번호 맞추기 (odsay API)

In [2]:
# 역 파일에서 이름 가져와서 검색
stations = pd.read_csv("result_train_station.csv")
stations

,역사명,노선명,주소,위도,경도,역번호,외부코드
0,까치울,7호선,경기도 부천시 원미구 길주로 지하 626 (춘의동),37.506130,126.810930,3753,751
1,부천종합운동장,7호선,경기도 부천시 원미구 길주로 지하 502 (춘의동),37.505020,126.796610,3754,752
2,춘의,7호선,경기도 부천시 원미구 길주로 지하 406 (춘의동),37.503650,126.788280,3755,753
3,신중동,7호선,경기도 부천시 원미구 길주로 지하 314 (중동),37.502820,126.775660,3756,754
4,부천시청,7호선,경기도 부천시 원미구 길주로 지하 202 (중동),37.504440,126.763640,3757,755
...,...,...,...,...,...,...,...
644,옥수,경의중앙선,서울시 성동구 동호로 지하21(옥수동),37.540446,127.018672,NaN,NaN
645,응봉,경의중앙선,서울시 성동구 고산자로 123(응봉동),37.549946,127.034538,NaN,NaN
646,까치산,2호선,서울특별시 강서구 강서로 54,37.531394,126.846987,0200,NaN
647,도라산,경의중앙선,경기도 파주시 장단면 희망로 307,37.898307,126.709193,NaN,NaN


In [3]:
# key 가져오기
from dotenv import load_dotenv
import os

load_dotenv()

odsay_url = os.environ.get('odsay_searchstation_url')
seoul_url = os.environ.get('train_seoul_now_url')

In [9]:
def mkStationNumFromAPI(stations) :
    err = list()
    result = pd.DataFrame([], columns=["노선명", "역사명", "역코드"])
    
    cnt = 0
    for i in range(len(stations)) :
        station = stations["역사명"].loc[i]
        
        ind = station.find("(")
        if ind != 1:
            # ( 뒤쪽 자르기
            station = station[:ind]
        
        response = urllib.request.urlopen(odsay_url.format(0, urllib.parse.quote(
            station, encoding='utf-8'), 2))
        json_str = response.read().decode("utf-8")
        json_obj = json.loads(json_str)
        
        # print(json_obj)
        if json_obj['result']['totalCount'] == 0:
            # 역 검색 실패, 문제 있는 case
            err.append((station, stations["노선명"].loc[i]))
        else:
            for j in range(json_obj['result']['totalCount']):
                nowObj = json_obj['result']['station'][j]
                result.loc[cnt] = [nowObj['laneName'], nowObj['stationName'], nowObj['stationID']]
                
                cnt += 1
                
    print(err)
    return result

In [10]:
stations_odsay = mkStationNumFromAPI(stations)

[('도라', '경의중앙선')]


In [11]:
stations_odsay

,노선명,역사명,역코드
0,수도권 2호선,까치산,264
1,수도권 5호선,까치산,518
2,수도권 7호선,까치울,751
3,수도권 7호선,부천종합운동장,752
4,수도권 7호선,춘의,753
...,...,...,...
4349,수도권 2호선,까치산,264
4350,수도권 5호선,까치산,518
4351,수도권 7호선,까치울,751
4352,수도권 1호선,광운대,120


In [12]:
print(set(stations_odsay["노선명"]))

{'부산 4호선', '수도권 2호선', '수도권 공항철도', '대구 3호선', '대구 2호선', '수도권 서해선(소사-원시)', '부산 1호선', '수도권 8호선', '광주 1호선', '수도권 경춘선', '수도권 신분당선', '경의중앙선', '부산 3호선', '수도권 에버라인', '대전 1호선', '신림선', '수도권 1호선', '수도권 5호선', '수도권 3호선', '인천 1호선', '수도권 의정부경전철', '수도권 7호선', '부산-김해경전철', '김포골드라인', '수도권 4호선', '수인.분당선', '부산 2호선', '대구 1호선', '수도권 6호선', '인천 2호선', '수도권 9호선', '우이신설경전철', '수도권 경강선', '동해선'}


In [14]:
stations_odsay = stations_odsay[stations_odsay["노선명"].str.slice(
    start=0, stop=2) != "부산"]
stations_odsay = stations_odsay[stations_odsay["노선명"].str.slice(
    start=0, stop=2) != "대구"]
stations_odsay = stations_odsay[stations_odsay["노선명"].str.slice(
    start=0, stop=2) != "광주"]
stations_odsay = stations_odsay[stations_odsay["노선명"].str.slice(
    start=0, stop=2) != "인천"]
stations_odsay = stations_odsay[stations_odsay["노선명"].str.slice(
    start=0, stop=2) != "대전"]
stations_odsay = stations_odsay[stations_odsay["노선명"] != "수도권 서해선(소사-원시)"]
stations_odsay = stations_odsay[stations_odsay["노선명"] != "수도권 에버라인"]
stations_odsay = stations_odsay[stations_odsay["노선명"] != "김포골드라인"]
stations_odsay = stations_odsay[stations_odsay["노선명"] != "동해선"]
stations_odsay = stations_odsay[stations_odsay["노선명"] != "수도권 의정부경전철"]
stations_odsay = stations_odsay[stations_odsay["노선명"] != "수도권 경강선"]

print(set(stations_odsay["노선명"]))

{'수도권 2호선', '수도권 공항철도', '수도권 6호선', '수도권 4호선', '수도권 8호선', '신림선', '수도권 경춘선', '수도권 5호선', '수도권 신분당선', '수도권 1호선', '수도권 9호선', '수인.분당선', '우이신설경전철', '수도권 7호선', '수도권 3호선', '경의중앙선'}


In [16]:
stations_odsay = stations_odsay.drop_duplicates(["노선명", "역사명"])
stations_odsay

# 딱 필요한 개수만큼의 역코드 얻어냄

,노선명,역사명,역코드
0,수도권 2호선,까치산,264
1,수도권 5호선,까치산,518
2,수도권 7호선,까치울,751
3,수도권 7호선,부천종합운동장,752
4,수도권 7호선,춘의,753
...,...,...,...
3897,경의중앙선,원덕,1301
3901,수도권 3호선,원흥,370
4033,수도권 9호선,언주,926
4180,경의중앙선,팔당,1308


In [17]:
stations_odsay.to_csv("odsay_station.csv")

##### 10-b. station 파일과 맞추기

In [15]:
# 역이름 맞추기
station = pd.read_csv("result_train_station.csv")
odsay = pd.read_csv("odsay_station.csv")

In [16]:
odsay["노선명"] = odsay["노선명"].replace({
    "우이신설경전철": "우이신설선",
    "수인.분당선": "수인분당선",
    "수도권 신분당선": "신분당선",
    "수도권 공항철도": "공항철도",
    "수도권 경춘선": "경춘선"
})

set(odsay["노선명"])

{'경의중앙선',
 '경춘선',
 '공항철도',
 '수도권 1호선',
 '수도권 2호선',
 '수도권 3호선',
 '수도권 4호선',
 '수도권 5호선',
 '수도권 6호선',
 '수도권 7호선',
 '수도권 8호선',
 '수도권 9호선',
 '수인분당선',
 '신림선',
 '신분당선',
 '우이신설선'}

In [30]:
station["노선명"] = station["노선명"].replace({
    "1호선": "수도권 1호선",
    "2호선": "수도권 2호선",
    "3호선": "수도권 3호선",
    "4호선": "수도권 4호선",
    "5호선": "수도권 5호선",
    "6호선": "수도권 6호선",
    "7호선": "수도권 7호선",
    "8호선": "수도권 8호선",
    "9호선": "수도권 9호선",
    "공항철도선": "공항철도"
})

set(station["노선명"])

{'경의중앙선',
 '경춘선',
 '공항철도',
 '수도권 1호선',
 '수도권 2호선',
 '수도권 3호선',
 '수도권 4호선',
 '수도권 5호선',
 '수도권 6호선',
 '수도권 7호선',
 '수도권 8호선',
 '수도권 9호선',
 '수인분당선',
 '신림선',
 '신분당선',
 '우이신설선'}

In [17]:
odsay_station_name = set(odsay["역사명"])
station_name = set(station["역사명"])

odsay_diff = sorted(odsay_station_name - station_name)
station_diff = sorted(station_name - odsay_station_name)

print(odsay_station_name)
print(station_name)
print(odsay_diff)
print(station_diff)

{'도원', '올림픽공원', '도화', '신용산', '배방', '팔당', '오빈', '선바위', '까치울', '구반포', '도봉', '백마', '잠실', '임진강', '가산디지털단지', '광교중앙(아주대)', '공항시장', '북한산우이', '산곡', '하남시청(덕풍.신장)', '송탄', '광교(경기대)', '명동', '오리', '월롱', '선유도', '까치산', '신방화', '삼송', '경찰병원', '문산', '용답', '상봉', '당고개', '충무로', '신이문', '상일동', '숭실대입구', '무악재', '삼전', '노들', '문래', '서울대벤처타운', '쌍문', '둔촌동', '대야미', '제기동', '구산', '한강진', '구로', '중화', '산성', '마곡', '연수', '탕정', '풍산', '한남', '잠원', '검암', '사당', '창동', '혜화', '암사', '이촌', '야목', '지행', '용마산', '둔촌오륜', '죽전', '신창(순천향대)', '한양대', '야탑', '금정', '몽촌토성', '정왕', '녹양', '역삼', '원인재', '중랑', '망포', '관악', '언주', '구파발', '남춘천', '양재시민의숲', '철산', '고색', '서현', '정발산', '상월곡', '구로디지털단지', '의왕', '백운', '도심', '사리', '보산', '신정', '마두', '압구정로데오', '양천향교', '공항화물청사', '계양', '주엽', '부평구청', '상갈', '마들', '동두천중앙', '서울역', '아차산', '과천', '남태령', '한티', '수색', '거여', '남영', '신길', '고덕', '잠실나루', '신답', '경마공원', '시청', '부천', '강남', '학여울', '구성', '봉명', '수내', '화서', '화전', '독바위', '화계', '먹골', '동대문', '범계', '석계', '동대문역사문화공원', '마천', '단대오거리', '광화문', '보문', '천안', '장한평', '소요산', '신목동', '원덕', '양재

In [18]:
station["역사명"] = station["역사명"].replace({
    '강변(동서울터미널)': '강변', 
    '경복궁(정부서울청사)': '경복궁', 
    '고려대(종암)': '고려대', 
    '공릉(서울과학기술대)': '공릉', 
    '광나루(장신대)': '광나루', 
    '광화문(세종문화회관)': '광화문', 
    '광흥창(서강)': '광흥창', 
    '교대(법원.검찰청)': '교대', 
    '구의(광진구청)': '구의', 
    '군자(능동)': '군자', 
    '굽은다리(강동구민회관앞)': '굽은다리', 
    '기흥(백남준아트센터)': '기흥', 
    '낙성대(강감찬)': '낙성대', 
    '남부터미널(예술의전당)': '남부터미널', 
    '남한산성입구(성남법원.검찰청)': '남한산성입구', 
    '대림(구로구청)': '대림', 
    '대흥(서강대앞)': '대흥', 
    '동대문역사문화공원(DDP)': '동대문역사문화공원', 
    '동작(현충원)': '동작', 
    '몽촌토성(평화의문)': '몽촌토성', 
    '미아(서울사이버대학)': '미아', 
    '삼성(무역센터)': '삼성', 
    '상봉(시외버스터미널)': '상봉', 
    '상월곡(한국과학기술연구원)': '상월곡', 
    '새절(신사)': '새절', 
    '서울대입구(관악구청)': '서울대입구', 
    '성신여대입구(돈암)': '성신여대입구', 
    '숙대입구(갈월)': '숙대입구', 
    '숭실대입구(살피재)': '숭실대입구', 
    '신정(은행정)': '신정', 
    '아차산(어린이대공원후문)': '아차산', 
    '안암(고대병원앞)': '안암', 
    '양재(서초구청)': '양재', 
    '양재시민의숲(매헌)': '양재시민의숲', 
    '어린이대공원(세종대)': '어린이대공원', 
    '오목교(목동운동장앞)': '오목교', 
    '온수(성공회대입구)': '온수', 
    '올림픽공원(한국체대)': '올림픽공원', 
    '왕십리(성동구청)': '왕십리', 
    '용두(동대문구청)': '용두', 
    '용마산(용마폭포공원)': '용마산', 
    '월곡(동덕여대)': '월곡', 
    '월드컵경기장(성산)': '월드컵경기장', 
    '이촌(국립중앙박물관)': '이촌', 
    '잠실(송파구청)': '잠실', 
    '종로3가(탑골공원)': '종로3가', 
    '증산(명지대앞)': '증산', 
    '천호(풍납토성)': '천호', 
    '청량리(서울시립대입구)': '청량리', 
    '충정로(경기대입구)': '충정로', 
    '한성대입구(삼선교)': '한성대입구', 
    '화랑대(서울여대입구)': '화랑대', 
    '회현(남대문시장)': '회현', 
    '흑석(중앙대입구)': '흑석'
})


In [19]:
odsay_station_name = set(odsay["역사명"])
station_name = set(station["역사명"])

odsay_diff = sorted(odsay_station_name - station_name)
station_diff = sorted(station_name - odsay_station_name)

print(odsay_station_name)
print(station_name)
print(odsay_diff)
print(station_diff)

{'도원', '올림픽공원', '도화', '신용산', '배방', '팔당', '오빈', '선바위', '까치울', '구반포', '도봉', '백마', '잠실', '임진강', '가산디지털단지', '광교중앙(아주대)', '공항시장', '북한산우이', '산곡', '하남시청(덕풍.신장)', '송탄', '광교(경기대)', '명동', '오리', '월롱', '선유도', '까치산', '신방화', '삼송', '경찰병원', '문산', '용답', '상봉', '당고개', '충무로', '신이문', '상일동', '숭실대입구', '무악재', '삼전', '노들', '문래', '서울대벤처타운', '쌍문', '둔촌동', '대야미', '제기동', '구산', '한강진', '구로', '중화', '산성', '마곡', '연수', '탕정', '풍산', '한남', '잠원', '검암', '사당', '창동', '혜화', '암사', '이촌', '야목', '지행', '용마산', '둔촌오륜', '죽전', '신창(순천향대)', '한양대', '야탑', '금정', '몽촌토성', '정왕', '녹양', '역삼', '원인재', '중랑', '망포', '관악', '언주', '구파발', '남춘천', '양재시민의숲', '철산', '고색', '서현', '정발산', '상월곡', '구로디지털단지', '의왕', '백운', '도심', '사리', '보산', '신정', '마두', '압구정로데오', '양천향교', '공항화물청사', '계양', '주엽', '부평구청', '상갈', '마들', '동두천중앙', '서울역', '아차산', '과천', '남태령', '한티', '수색', '거여', '남영', '신길', '고덕', '잠실나루', '신답', '경마공원', '시청', '부천', '강남', '학여울', '구성', '봉명', '수내', '화서', '화전', '독바위', '화계', '먹골', '동대문', '범계', '석계', '동대문역사문화공원', '마천', '단대오거리', '광화문', '보문', '천안', '장한평', '소요산', '신목동', '원덕', '양재

In [21]:
station[station["역사명"] == "총신대입구(이수)"]

# 7호선 역사명 수동 변경
# 도라산은 없어도 무관 (시간표에도 없음, 서울 막차와 무관)

,역사명,노선명,주소,위도,경도,역번호,외부코드
185,총신대입구(이수),4호선,서울특별시 동작구 동작대로 지하117(사당동),37.486263,126.981989,0432,432
310,총신대입구(이수),7호선,서울특별시 동작구 사당로 지하310(사당동),37.485196,126.981605,2738,736


In [31]:
result = pd.merge(station, odsay, on=['역사명', '노선명'], how='outer')

result

,역사명,노선명,주소,위도,경도,역번호,외부코드,Unnamed: 0,역코드
0,까치울,수도권 7호선,경기도 부천시 원미구 길주로 지하 626 (춘의동),37.506130,126.810930,3753,751,2.0,751.0
1,부천종합운동장,수도권 7호선,경기도 부천시 원미구 길주로 지하 502 (춘의동),37.505020,126.796610,3754,752,3.0,752.0
2,춘의,수도권 7호선,경기도 부천시 원미구 길주로 지하 406 (춘의동),37.503650,126.788280,3755,753,4.0,753.0
3,신중동,수도권 7호선,경기도 부천시 원미구 길주로 지하 314 (중동),37.502820,126.775660,3756,754,6.0,754.0
4,부천시청,수도권 7호선,경기도 부천시 원미구 길주로 지하 202 (중동),37.504440,126.763640,3757,755,7.0,755.0
...,...,...,...,...,...,...,...,...,...
645,응봉,경의중앙선,서울시 성동구 고산자로 123(응봉동),37.549946,127.034538,NaN,NaN,1514.0,193.0
646,까치산,수도권 2호선,서울특별시 강서구 강서로 54,37.531394,126.846987,0200,NaN,0.0,264.0
647,도라산,경의중앙선,경기도 파주시 장단면 희망로 307,37.898307,126.709193,NaN,NaN,NaN,NaN
648,광운대,경춘선,서울특별시 노원구 석계로 98-2,37.623986,127.061956,1305,NaN,2310.0,1809.0


In [32]:
result.to_csv("result_train_station_withcode.csv")

### 11. 역이름 맞추기

##### 11-a. 역이름 API로부터 가져오기

In [7]:
result = pd.read_csv('result_train_station_withcode.csv')

In [3]:


seoul_API = pd.read_csv("train_station_API_seoul.csv")
seoul_API.rename(columns={
    'STATN_NM': '역사명',
    '호선이름': '노선명'
}, inplace=True)

seoul_API

,SUBWAY_ID,STATN_ID,역사명,노선명
0,1001,1001000100,소요산,1호선
1,1001,1001000101,동두천,1호선
2,1001,1001000102,보산,1호선
3,1001,1001000103,동두천중앙,1호선
4,1001,1001000104,지행,1호선
...,...,...,...,...
627,1092,1092004709,북한산보국문,우이신설선
628,1092,1092004710,정릉,우이신설선
629,1092,1092004711,성신여대입구,우이신설선
630,1092,1092004712,보문,우이신설선


In [8]:
seoul_API["노선명"] = seoul_API["노선명"].replace({
    "1호선": "수도권 1호선",
    "2호선": "수도권 2호선",
    "3호선": "수도권 3호선",
    "4호선": "수도권 4호선",
    "5호선": "수도권 5호선",
    "6호선": "수도권 6호선",
    "7호선": "수도권 7호선",
    "8호선": "수도권 8호선",
    "9호선": "수도권 9호선",
    "경의중앙선(1호선)": "경의중앙선",
    "경춘선(1호선)": "경춘선"
})

set(seoul_API["노선명"])

{'경의중앙선',
 '경춘선',
 '공항철도',
 '수도권 1호선',
 '수도권 2호선',
 '수도권 3호선',
 '수도권 4호선',
 '수도권 5호선',
 '수도권 6호선',
 '수도권 7호선',
 '수도권 8호선',
 '수도권 9호선',
 '수인분당선',
 '신분당선',
 '우이신설선'}

In [9]:
result_station_name = set(result["역사명"])
seoul_name = set(seoul_API["역사명"])

result_diff = sorted(result_station_name - seoul_name)
seoul_diff = sorted(seoul_name - result_station_name)

print(result_station_name)
print(seoul_name)
print(result_diff)
print(seoul_diff)

{'홍제', '진접', '인천논현', '신정네거리', '상계', '신당', '사평', '개포동', '오목천', '호구포', '철산', '대야미', '세마', '임진강', '수진', '마포구청', '수지구청', '도라산', '디지털미디어시티', '영등포구청', '선릉', '동천', '마곡나루', '구산', '원당', '남동인더스파크', '신사', '동암', '마석', '신포', '녹번', '소래포구', '대화', '마장', '행당', '모란', '부평구청', '상월곡', '지평', '남한산성입구', '뚝섬유원지', '등촌', '아차산', '탕정', '운정', '강남구청', '수내', '야목', '중앙보훈병원', '망우', '역곡', '노원', '운길산', '마곡', '수원', '청명', '도봉산', '금정', '강남', '잠실새내', '남영', '도원', '마두', '신흥', '신길온천', '춘의', '양재', '학여울', '양원', '효창공원앞', '백운', '군자', '도곡', '방이', '상갈', '동두천', '상현', '관악', '신방화', '약수', '매탄권선', '개봉', '수리산', '서초', '잠실', '소사', '혜화', '장한평', '대방', '상동', '한티', '용두', '압구정로데오', '명학', '가락시장', '병점', '잠원', '명동', '도농', '흑석', '동작', '일산', '연수', '인천', '신목동', '삼성', '개화', '문산', '굴봉산', '한대앞', '광명', '독립문', '합정', '을지로4가', '부개', '오빈', '용산', '기흥', '사당', '까치산', '삼전', '당고개', '신대방삼거리', '구성', '퇴계원', '오류동', '야당', '종로5가', '먹골', '용답', '청량리', '금호', '충정로', '상일동', '노량진', '의정부', '선바위', '뚝섬', '한남', '평내호평', '서동탄', '금촌', '마들', '능곡', '파주', '서울대입구', '신대방', '성신여대입구', '서대문'

In [11]:
result["역사명"] = result["역사명"].replace({
    '4.19민주묘지': '4.19 민주묘지', 
    '광교(경기대)': '광교', 
    '광교중앙(아주대)': '광교중앙', 
    '광나루': '광나루(장신대)', 
    '녹사평(용산구청)': '녹사평', 
    '몽촌토성': '몽촌토성(평화의문)', 
    '봉화산(서울의료원)': '봉화산', 
    '서울역': '서울', 
    '석남(거북시장)': '석남', 
    '수유(강북구청)': '수유', 
    '신창(순천향대)': '신창', 
    '아차산': '아차산(어린이대공원후문)', 
    '어린이대공원': '어린이대공원(세종대)', 
    '천호': '천호(풍납토성)', 
    '하남시청(덕풍.신장)': '하남시청'
})

In [12]:
result_station_name = set(result["역사명"])
seoul_name = set(seoul_API["역사명"])

result_diff = sorted(result_station_name - seoul_name)
seoul_diff = sorted(seoul_name - result_station_name)

print(result_station_name)
print(seoul_name)
print(result_diff)
print(seoul_diff)

{'홍제', '진접', '인천논현', '신정네거리', '상계', '신당', '사평', '개포동', '오목천', '호구포', '철산', '대야미', '세마', '임진강', '수진', '마포구청', '수지구청', '도라산', '디지털미디어시티', '영등포구청', '선릉', '동천', '마곡나루', '구산', '원당', '남동인더스파크', '신사', '동암', '마석', '신포', '녹번', '소래포구', '대화', '마장', '행당', '모란', '부평구청', '상월곡', '지평', '남한산성입구', '신창', '뚝섬유원지', '등촌', '탕정', '운정', '강남구청', '수내', '야목', '중앙보훈병원', '망우', '역곡', '노원', '운길산', '마곡', '수원', '청명', '도봉산', '금정', '강남', '잠실새내', '남영', '도원', '마두', '신흥', '신길온천', '춘의', '양재', '학여울', '양원', '효창공원앞', '백운', '군자', '도곡', '방이', '상갈', '동두천', '상현', '관악', '신방화', '약수', '매탄권선', '개봉', '수리산', '서초', '잠실', '소사', '혜화', '장한평', '대방', '상동', '한티', '용두', '아차산(어린이대공원후문)', '압구정로데오', '명학', '가락시장', '병점', '잠원', '명동', '도농', '흑석', '동작', '일산', '연수', '인천', '신목동', '삼성', '개화', '문산', '굴봉산', '한대앞', '광명', '독립문', '합정', '을지로4가', '부개', '오빈', '용산', '기흥', '사당', '까치산', '삼전', '당고개', '신대방삼거리', '구성', '퇴계원', '오류동', '야당', '종로5가', '먹골', '용답', '청량리', '금호', '충정로', '상일동', '노량진', '의정부', '선바위', '뚝섬', '한남', '평내호평', '서동탄', '금촌', '마들', '능곡', '파주', '서울대입구', '신대방',

In [13]:
result[result["역사명"] == "군자"]

,Unnamed: 0.1,역사명,노선명,주소,위도,경도,역번호,외부코드,Unnamed: 0,역코드
222,222,군자,수도권 5호선,서울특별시 광진구 천호대로 지하550(능동),37.557088,127.079577,2545,544,1402.0,544.0
299,299,군자,수도권 7호선,서울특별시 광진구 천호대로 지하550(능동),37.556897,127.079338,2727,725,1403.0,725.0


In [15]:
result["역사명"] = result["역사명"].replace({
    '군자': '군자(능동)',
    '이수': '총신대입구(이수)'
})

# 그 외, 실제 역이름과 다르게 검색해야 하는 부분은 코드상에서 처리 필요
# 응암 -> 응암순환(상선), 평택지제 -> 지제, 서울역 -> 서울
# * 서울시 이외의 역구간은 미제공 됩니다.(예, 광명, 서동탄, 춘천 등)

# * 주의사항: 원천에서 데이터가 수집 및 가공되어 서비스되는 과정에서 시간 차가 발생할 수 있습니다.
# 출력값 중 recptnDt(열차 도착정보를 생성한 시각)는 데이터가 생성된 시간을 의미하며 현재시각과 recptnDt의 차이 만큼 열차가 더 진행한 것으로 보정해서 사용해야 합니다.
# 예시) 현재시간이 10시 5분 30초이고, recptnDt가 10시 3분 30초인경우 2분간의 시차가 발생하므로 도착정보는 2분씩 당겨지거나 1개의 역을 더 진행한것으로 판단

In [17]:
# 확인해보기 

temp = pd.merge(result, seoul_API, on=['역사명', '노선명'], how='left')

temp

,Unnamed: 0.1,역사명,노선명,주소,위도,경도,역번호,외부코드,Unnamed: 0,역코드,SUBWAY_ID,STATN_ID
0,0,까치울,수도권 7호선,경기도 부천시 원미구 길주로 지하 626 (춘의동),37.506130,126.810930,3753,751,2.0,751.0,1007.0,1.007001e+09
1,1,부천종합운동장,수도권 7호선,경기도 부천시 원미구 길주로 지하 502 (춘의동),37.505020,126.796610,3754,752,3.0,752.0,1007.0,1.007001e+09
2,2,춘의,수도권 7호선,경기도 부천시 원미구 길주로 지하 406 (춘의동),37.503650,126.788280,3755,753,4.0,753.0,1007.0,1.007001e+09
3,3,신중동,수도권 7호선,경기도 부천시 원미구 길주로 지하 314 (중동),37.502820,126.775660,3756,754,6.0,754.0,1007.0,1.007001e+09
4,4,부천시청,수도권 7호선,경기도 부천시 원미구 길주로 지하 202 (중동),37.504440,126.763640,3757,755,7.0,755.0,1007.0,1.007001e+09
...,...,...,...,...,...,...,...,...,...,...,...,...
645,645,응봉,경의중앙선,서울시 성동구 고산자로 123(응봉동),37.549946,127.034538,NaN,NaN,1514.0,193.0,1063.0,1.063075e+09
646,646,까치산,수도권 2호선,서울특별시 강서구 강서로 54,37.531394,126.846987,0200,NaN,0.0,264.0,1002.0,1.002002e+09
647,647,도라산,경의중앙선,경기도 파주시 장단면 희망로 307,37.898307,126.709193,NaN,NaN,NaN,NaN,NaN,NaN
648,648,광운대,경춘선,서울특별시 노원구 석계로 98-2,37.623986,127.061956,1305,NaN,2310.0,1809.0,1067.0,1.067080e+09


In [18]:
temp[pd.isnull(temp["SUBWAY_ID"])]

,Unnamed: 0.1,역사명,노선명,주소,위도,경도,역번호,외부코드,Unnamed: 0,역코드,SUBWAY_ID,STATN_ID
54,54,샛강,신림선,서울시 영등포구 여의도동 56,37.517097,126.929399,4401,NaN,295.0,11711.0,NaN,NaN
55,55,대방,신림선,서울시 영등포구 신길동 1376-1,37.513306,126.925726,4402,NaN,303.0,11712.0,NaN,NaN
56,56,서울지방병무청,신림선,서울시 동작구 대방동 466-41,37.506046,126.922708,4403,NaN,40.0,11713.0,NaN,NaN
57,57,보라매,신림선,서울시 동작구 대방동 466-49,37.500274,126.920435,4404,NaN,133.0,11714.0,NaN,NaN
58,58,보라매공원,신림선,서울시 동작구 신대방동 722,37.495569,126.918083,4405,NaN,134.0,11715.0,NaN,NaN
59,59,보라매병원,신림선,서울시 동작구 신대방동 425,37.492960,126.923496,4406,NaN,135.0,11716.0,NaN,NaN
60,60,당곡,신림선,서울시 관악구 봉천동 985,37.490300,126.927513,4407,NaN,332.0,11717.0,NaN,NaN
61,61,신림,신림선,서울시 관악구 신림동 1467-10,37.484927,126.929616,4408,NaN,163.0,11718.0,NaN,NaN
62,62,서원,신림선,서울시 관악구 신림동 808-717,37.478234,126.933037,4409,NaN,401.0,11719.0,NaN,NaN
63,63,서울대벤처타운,신림선,서울시 관악구 신림동 110-99,37.472002,126.933935,4410,NaN,33.0,11720.0,NaN,NaN


In [19]:
# 지도 시각화, 문제되는지 확인
geo_station = temp[pd.isnull(temp["SUBWAY_ID"])]

# 중심 설정
fmap = folium.Map(location=[geo_station['위도'].mean(
), geo_station['경도'].mean()], zoom_start=8, width=750, height=500)

for n in geo_station.index:
    # 팝업에 들어갈 텍스트를 지정
    popup_name = str(geo_station.loc[n, '위도']) + ", " + str(geo_station.loc[n, '경도']) + \
        " \n" + geo_station.loc[n, '역사명'] + ' - ' + geo_station.loc[n, '노선명']

    folium.Marker(
        location=[geo_station.loc[n, '위도'], geo_station.loc[n, '경도']],
        popup=popup_name,
        tooltip=popup_name
    ).add_to(fmap)

fmap

# 확인 결과, 신림선 제외 다 이름으로 실시간 검색 가능할 것 같음

In [20]:
# key 가져오기
from dotenv import load_dotenv
import os

load_dotenv()

url = os.environ.get('train_seoul_realtime_station_url')


In [21]:
# API 호출 테스트
# 안되는 역 종류 확인해보기
err = dict()

for i in range(len(result["역사명"])):
    name = result["역사명"].loc[i]
    route = result["노선명"].loc[i]
    address = result["주소"].loc[i]
    
    response = urllib.request.urlopen(url.format(urllib.parse.quote(
        name, encoding='utf-8'), 2))
    json_str = response.read().decode("utf-8")
    json_obj = json.loads(json_str)
    
    # print(json_obj)

    if 'code' in json_obj and json_obj['code'] != 'INFO-000':
        err[str(name) + " - " + str(route) + " - " + str(address)] = json_obj['code']
    elif 'errorMessage' in json_obj and json_obj['errorMessage']['code'] != 'INFO-000':
        err[str(name) + " - " + str(route) + " - " + str(address)] = json_obj['errorMessage']['code']

In [22]:
err

{'북한산우이 - 우이신설선 - 서울특별시 강북구 삼양로 지하 676': 'INFO-200',
 '서울지방병무청 - 신림선 - 서울시 동작구 대방동 466-41': 'INFO-200',
 '보라매공원 - 신림선 - 서울시 동작구 신대방동 722': 'INFO-200',
 '보라매병원 - 신림선 - 서울시 동작구 신대방동 425': 'INFO-200',
 '당곡 - 신림선 - 서울시 관악구 봉천동 985': 'INFO-200',
 '서원 - 신림선 - 서울시 관악구 신림동 808-717': 'INFO-200',
 '서울대벤처타운 - 신림선 - 서울시 관악구 신림동 110-99': 'INFO-200',
 '관악산(서울대) - 신림선 - 서울시 관악구 신림동 211-10': 'INFO-200',
 '별내별가람 - 수도권 4호선 - 경기도 남양주시 별내3로 지하 345': 'INFO-200',
 '오남 - 수도권 4호선 - 경기도 남양주시 오남읍 진건오남로 지하 929': 'INFO-200',
 '진접 - 수도권 4호선 - 경기도 남양주시 진접읍 경복대로 지하 244': 'INFO-200',
 '신정 - 수도권 5호선 - 서울특별시 양천구 오목로 지하179(신정동)': 'INFO-200',
 '오목교 - 수도권 5호선 - 서울특별시 양천구 오목로 지하342(목동)': 'INFO-200',
 '굽은다리 - 수도권 5호선 - 서울특별시 강동구 양재대로 지하1572(명일동)': 'INFO-200',
 '강일 - 수도권 5호선 - 서울특별시 강동구 고덕로 지하456(강일동)': 'INFO-200',
 '미사 - 수도권 5호선 - 경기도 하남시 미사강변동로 지하90(망월동)': 'INFO-200',
 '하남풍산 - 수도권 5호선 - 경기도 하남시 덕풍서로 지하50(덕풍동)': 'INFO-200',
 '하남시청 - 수도권 5호선 - 경기도 하남시 하남대로 지하820(덕풍동)': 'INFO-200',
 '하남검단산 - 수도권 5호선 - 경기도 하남시 대청로 지하100(창우동)':

### 12. 세부 사항 변경

In [23]:
result

,Unnamed: 0.1,역사명,노선명,주소,위도,경도,역번호,외부코드,Unnamed: 0,역코드
0,0,까치울,수도권 7호선,경기도 부천시 원미구 길주로 지하 626 (춘의동),37.506130,126.810930,3753,751,2.0,751.0
1,1,부천종합운동장,수도권 7호선,경기도 부천시 원미구 길주로 지하 502 (춘의동),37.505020,126.796610,3754,752,3.0,752.0
2,2,춘의,수도권 7호선,경기도 부천시 원미구 길주로 지하 406 (춘의동),37.503650,126.788280,3755,753,4.0,753.0
3,3,신중동,수도권 7호선,경기도 부천시 원미구 길주로 지하 314 (중동),37.502820,126.775660,3756,754,6.0,754.0
4,4,부천시청,수도권 7호선,경기도 부천시 원미구 길주로 지하 202 (중동),37.504440,126.763640,3757,755,7.0,755.0
...,...,...,...,...,...,...,...,...,...,...
645,645,응봉,경의중앙선,서울시 성동구 고산자로 123(응봉동),37.549946,127.034538,NaN,NaN,1514.0,193.0
646,646,까치산,수도권 2호선,서울특별시 강서구 강서로 54,37.531394,126.846987,0200,NaN,0.0,264.0
647,647,도라산,경의중앙선,경기도 파주시 장단면 희망로 307,37.898307,126.709193,NaN,NaN,NaN,NaN
648,648,광운대,경춘선,서울특별시 노원구 석계로 98-2,37.623986,127.061956,1305,NaN,2310.0,1809.0


In [24]:
result.drop(columns=["Unnamed: 0.1", "역번호", "외부코드", "Unnamed: 0"], inplace=True)
result

,역사명,노선명,주소,위도,경도,역코드
0,까치울,수도권 7호선,경기도 부천시 원미구 길주로 지하 626 (춘의동),37.506130,126.810930,751.0
1,부천종합운동장,수도권 7호선,경기도 부천시 원미구 길주로 지하 502 (춘의동),37.505020,126.796610,752.0
2,춘의,수도권 7호선,경기도 부천시 원미구 길주로 지하 406 (춘의동),37.503650,126.788280,753.0
3,신중동,수도권 7호선,경기도 부천시 원미구 길주로 지하 314 (중동),37.502820,126.775660,754.0
4,부천시청,수도권 7호선,경기도 부천시 원미구 길주로 지하 202 (중동),37.504440,126.763640,755.0
...,...,...,...,...,...,...
645,응봉,경의중앙선,서울시 성동구 고산자로 123(응봉동),37.549946,127.034538,193.0
646,까치산,수도권 2호선,서울특별시 강서구 강서로 54,37.531394,126.846987,264.0
647,도라산,경의중앙선,경기도 파주시 장단면 희망로 307,37.898307,126.709193,NaN
648,광운대,경춘선,서울특별시 노원구 석계로 98-2,37.623986,127.061956,1809.0


In [25]:
result[pd.isnull(result["역코드"])]

# 추후 둘은 수동으로 넣어줄 것

,역사명,노선명,주소,위도,경도,역코드
310,총신대입구(이수),수도권 7호선,서울특별시 동작구 사당로 지하310(사당동),37.485196,126.981605,NaN
647,도라산,경의중앙선,경기도 파주시 장단면 희망로 307,37.898307,126.709193,NaN


In [27]:
result.to_csv('result_train_station_withcode.csv', index=False, encoding="utf-8-sig")

In [29]:
result = pd.read_csv('result_train_station_withcode.csv')
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648 entries, 0 to 647
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   역사명     648 non-null    object 
 1   노선명     648 non-null    object 
 2   주소      648 non-null    object 
 3   위도      648 non-null    float64
 4   경도      648 non-null    float64
 5   역코드     648 non-null    float64
dtypes: float64(3), object(3)
memory usage: 30.5+ KB
